In [ ]:
import zipfile

zip_path = "NuevoClasificadorImagenesClima/Backend/model/weather-dataset.zip"
extract_path = "NuevoClasificadorImagenesClima/Backend/dataset"  # Carpeta donde se descomprimirá


# Descomprimir el archivo
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Dataset descomprimido en: {extract_path}")


FileNotFoundError: [Errno 2] No such file or directory: 'Backend/model/weather-dataset.zip'

In [ ]:
import os

dataset_dir = "Backend/dataset"
for root, dirs, files in os.walk(dataset_dir):
    for file in files:
        print(os.path.join(root, file))

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from collections import Counter

In [ ]:
# Directorio del Dataset
data_dir = 'dataset'

In [ ]:
# Listar las carpetas (clases)
classes = os.listdir(data_dir)
print(f"Clases encontradas: {classes}")

In [ ]:
# Contar el número de imágenes por clase
image_count = {
    cls: len(os.listdir(os.path.join(data_dir, cls)))
    for cls in classes
}
print(f"Imágenes por clase: {image_count}")

In [ ]:
# Visualizar distribución de imágenes por clase
plt.bar(image_count.keys(), image_count.values())
plt.title('Distribución de Imágenes por Clase')
plt.xlabel('Clase')
plt.ylabel('Número de Imágenes')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Mostrar imágenes al azar
def show_random_images(data_dir, classes, num_images=5):
  fig, axes = plt.subplots(1, num_images, figsize=(15, 15))

  for ax in axes:
    cls = np.random.choice(classes)
    img_path = os.path.join(data_dir, cls, np.random.choice(os.listdir(os.path.join(data_dir, cls))))
    img = Image.open(img_path)
    ax.imshow(img)
    ax.set_title(cls)
    ax.axis('off')

  plt.show()

show_random_images(data_dir, classes)

In [ ]:
import shutil
from sklearn.model_selection import train_test_split

In [ ]:
# Establecer porcentajes para los conjuntos
validation_split = 0.15
test_split = 0.15
train_split = 1 - validation_split - test_split

# Configuración de directorios
train_dir = os.path.join(data_dir, 'train')
valid_dir = os.path.join(data_dir, 'valid')
test_dir = os.path.join(data_dir, 'test')

# Crear directorios de salida
os.makedirs(train_dir, exist_ok=True)
os.makedirs(valid_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

In [ ]:
# Procesar cada clase
for cls in classes:
    cls_dir = os.path.join(data_dir, cls)

    # Ignorar carpetas que no sean clases
    if not os.path.isdir(cls_dir):
        continue

    # Obtener imágenes de la clase
    images = os.listdir(cls_dir)

    # Separar en entrenamiento, validación y prueba
    try:
      train_images, temp_images = train_test_split(images, test_size=(validation_split + test_split), random_state=42)
      valid_images, test_images = train_test_split(temp_images, test_size=(test_split / (validation_split + test_split)), random_state=42)
    except ValueError as e:
      print(f"Error al dividir las imágenes de la clase '{cls}': {e}")
      continue

    # Crear subcarpetas para la clase en cada conjunto
    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(valid_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(test_dir, cls), exist_ok=True)

    # Mover imágenes a los conjuntos correspondientes
    for image in train_images:
        shutil.move(os.path.join(cls_dir, image), os.path.join(train_dir, cls, image))

    for image in valid_images:
        shutil.move(os.path.join(cls_dir, image), os.path.join(valid_dir, cls, image))

    for image in test_images:
        shutil.move(os.path.join(cls_dir, image), os.path.join(test_dir, cls, image))

print("Conjuntos creados con éxito:")
print(f"- Entrenamiento: {train_dir}")
print(f"- Validación: {valid_dir}")
print(f"- Prueba: {test_dir}")

In [ ]:
# Verificar las cantidades en cada conjunto
def count_images_in_directory(directory):
    total = 0
    for cls in os.listdir(directory):
        total += len(os.listdir(os.path.join(directory, cls)))
    return total

print(f"Imágenes en entrenamiento: {count_images_in_directory(train_dir)}")
print(f"Imágenes en validación: {count_images_in_directory(valid_dir)}")
print(f"Imágenes en prueba: {count_images_in_directory(test_dir)}")


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Preprocesamiento
data_gen = ImageDataGenerator(
    rescale=1./255,           # Normalización
    rotation_range=30,        # Aumento: rotaciones
    width_shift_range=0.2,    # Desplazamientos horizontales
    height_shift_range=0.2,   # Desplazamientos verticales
    zoom_range=0.2,           # Zoom aleatorio
    horizontal_flip=True,     # Volteo horizontal
    #validation_split=0.15     # Separación de validación
)

In [ ]:
# Crear generadores de datos
train_gen = data_gen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
# Generadores de validación y prueba (sin aumento)
valid_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    valid_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    test_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

In [ ]:
# Crear el modelo
model = Sequential([
    # Primera capa convolucional + pooling
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    # Segunda capa convolucional + pooling
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    # Tercera capa convolucional + pooling
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    # Primera capa convolucional adicional
    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    # Aplanar y añadir capas densas
    Flatten(),

    # Primera capa densa
    Dense(256, activation='relu'),
    Dropout(0.5),  # Regularización para evitar sobreajuste

    # Segunda capa densa
    Dense(128, activation='relu'),
    Dropout(0.5),  # Regularización para evitar sobreajuste

    # Capa de salida
    Dense(len(classes), activation='softmax')
])

In [ ]:
# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=30,
    steps_per_epoch=train_gen.samples // train_gen.batch_size,
    validation_steps=valid_gen.samples // valid_gen.batch_size
)

In [ ]:
# Evaluar el modelo con el conjunto de prueba
test_loss, test_accuracy = model.evaluate(test_gen, steps=test_gen.samples // test_gen.batch_size)

print(f"Precisión en el conjunto de prueba: {test_accuracy * 100:.2f}%")
print(f"Pérdida en el conjunto de prueba: {test_loss:.4f}")

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

In [ ]:
# Obtener predicciones
test_gen.reset()
predictions = model.predict(test_gen, steps=len(test_gen), verbose=1)
predicted_classes = np.argmax(predictions, axis=1)

# Obtener etiquetas verdaderas
true_classes = test_gen.classes
class_labels = list(test_gen.class_indices.keys())

# Matriz de confusión
confusion_mtx = confusion_matrix(true_classes, predicted_classes)

# Visualizar matriz de confusión
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.title('Matriz de Confusión')
plt.ylabel('Clase Real')
plt.xlabel('Clase predicha')
plt.show()

# Reporte de clasificación
print("\nReporte de Clasificación:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

In [ ]:
# prompt: Make the ROC curv graphic for all classes

from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt

# Binarizar las etiquetas
y_true_bin = label_binarize(true_classes, classes=np.arange(len(class_labels)))

# Calcular las curvas ROC para cada clase
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(len(class_labels)):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], predictions[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Graficar las curvas ROC
plt.figure(figsize=(10, 8))
for i in range(len(class_labels)):
    plt.plot(fpr[i], tpr[i], label=f'{class_labels[i]} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--')  # Línea de referencia
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curvas ROC para cada clase')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Calculate micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_true_bin.ravel(), predictions.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Plot micro-average ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]))

plt.plot([0, 1], [0, 1], 'k--')  # Línea diagonal
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Micro-average Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Especificar la ruta y el nombre del archivo donde se guardará el modelo
model_path = 'model/climate_classifier.h5'

# Guardar el modelo en formato HDF5 (.h5)
model.save(model_path)

print(f"Modelo guardado exitosamente en {model_path}")
